In [2]:
! pip install insightface
! pip install onnxruntime

  Using cached insightface-0.7.3-cp38-cp38-linux_x86_64.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 43.1 MB/s eta 0:00:0000:0100:01
  Using cached matplotlib-3.7.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.2 MB)
  Using cached scikit_learn-1.2.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (9.8 MB)
  Using cached scikit_image-0.20.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.4 MB)
  Using cached Cython-0.29.34-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (2.0 MB)
  Using cached albumentations-1.3.0-py3-none-any.whl (123 kB)
  Using cached qudida-0.0.4-py3-none-any.whl (3.5 kB)
  Using cached contourpy-1.0.7-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.7 MB/s eta 0:00:00
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '

In [1]:
import insightface
import numpy as np
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image
import cv2
import os
import csv

In [15]:
def distance(embeddings1, embeddings2, distance_type='Cosine'):
    embeddings1=embeddings1.astype(np.float64)
    embeddings2=embeddings2.astype(np.float64)
    if distance_type=='Euclidian':
        # Euclidian distance
        embeddings1 = embeddings1/np.linalg.norm(embeddings1, axis=0, keepdims=True)
        embeddings2 = embeddings2/np.linalg.norm(embeddings2, axis=0, keepdims=True)
        diff = np.subtract(embeddings1, embeddings2)
        dist = np.sum(np.square(diff),0)
    elif distance_type=='Cosine':
        # Distance based on cosine similarity
        dot = np.sum(np.multiply(embeddings1, embeddings2), axis=0)
        norm = np.linalg.norm(embeddings1, axis=0) * np.linalg.norm(embeddings2, axis=0)
        similarity = dot/norm
        similarity = min(1,similarity)
        dist=1-similarity
    else:
        raise 'Undefined distance metric %d' % distance_metric 
    return dist

In [3]:
fr_model = insightface.app.FaceAnalysis()
ctx_id = 0
fr_model.prepare(ctx_id = ctx_id, det_thresh=0.1)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/ubuntu/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
se

In [4]:
img1 = cv2.imread("/volumes/1/subjects/04832d112.jpg") #SUBJECT IMAGE
img1_mated = cv2.imread("/volumes/1/output/frgc_cluster_10.png") #MATED (CLUSTER IMAGE)
img2 = cv2.imread("/volumes/1/thomas/thomas-comparison.jpg") #NON-MATED

face1 = fr_model.get(img1)
face1_mated = fr_model.get(img1_mated)
face2 = fr_model.get(img2)

embedding1 = face1[0].normed_embedding
embedding1_mated = face1_mated[0].normed_embedding
embedding2 = face2[0].normed_embedding

/home/ubuntu/environments/my_env/lib/python3.8/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


In [16]:
print(f"Mated Comparison Score: {distance(embedding1, embedding2)}")
print(f"Non-Mated Comparison Score: {distance(embedding1, embedding1_mated)}")

Mated Comparison Score: 0.9208778034582
Non-Mated Comparison Score: 0.9638482064180096


In [24]:
import os
import cv2
import csv

# Update the path to the FRGC results file
results_file_path = '/home/ubuntu/trainingfiles/frgc_results_cluster_id_to_subjects.txt'
output_file_path = '/home/ubuntu/trainingfiles/all_comparison_scores.csv'  # Path to the output CSV file


with open(results_file_path, 'r') as f:
    reader = csv.reader(f, delimiter=':')
    clusters_dict = {}
    for i, row in enumerate(reader):
        if i < 2:  # Skip the first two rows
            continue
        cluster_num = int(row[0].strip())
        user_ids = [user_id.strip()[1:-1] for user_id in row[1].strip()[1:-1].split(',')]
        clusters_dict[cluster_num] = user_ids
        
num_higher = 0 # variable to keep track of how many times non-mated comparison score is higher
total_comparisons = 0 # variable to keep track of total number of comparisons


excel_scores = []  # List to store mated comparison scores

# Loop through the clusters in the dictionary and process the images
for cluster_num, user_ids in clusters_dict.items():
    print(f"Processing cluster {cluster_num} with user IDs {user_ids}")
    for user_id in user_ids:
        print("Processing user ID: ", user_id)
        # Find all images in the folder that match the user_id 
        img_filenames = sorted([filename for filename in os.listdir('/volumes/1/aligned/') if filename.startswith(user_id) and filename.endswith('.jpg')])
        
        # If there are no matching images, skip to the next user_id
        if not img_filenames:
            print("No images found for user ID:", user_id)
            continue
        
        # Load the first image for this user_id
        img_filename = img_filenames[0]
        img_path = os.path.join('/volumes/1/aligned/', img_filename)
        img = cv2.imread(img_path)
        
        # Load the second image for this user_id
        img_mated_filename = img_filenames[1]
        img_mated_path = os.path.join('/volumes/1/aligned/', img_mated_filename)
        img_mated = cv2.imread(img_mated_path)
        
        img_gan_filename = 'frgc_cluster_{}.png'.format(cluster_num)
        img_gan_path = os.path.join('/volumes/1/output/', img_gan_filename) 
        img_gan = cv2.imread(img_gan_path)
        
        img2_filename = 'thomas-comparison.jpg'
        img2_path = os.path.join('/volumes/1/thomas/', img2_filename)
        img2 = cv2.imread(img2_path)
        
        face = fr_model.get(img)
        face_gan = fr_model.get(img_gan)
        face_mated = fr_model.get(img_mated)

        
        # Try to get the face embedding for img2
        try:
            face2 = fr_model.get(img2)
            
            embedding = face[0].normed_embedding
            embedding_gan = face_gan[0].normed_embedding
            embedding_mated = face_mated[0].normed_embedding
            embedding2 = face2[0].normed_embedding
            mated_score = distance(embedding, embedding_mated)
            non_mated_score = distance(embedding, embedding2)
            gan_score = distance(embedding, embedding_gan)
            if non_mated_score > gan_score:
                scoring = "GAN"
            elif non_mated_score < gan_score:
                scoring = "Non-Mated"
            print(f"Non-Mated Comparison Score: {non_mated_score}")
            print(f"GAN Comparison Score: {gan_score}")
            print(f"Mated Comparison Score: {mated_score}")
            excel_scores.append([cluster_num,user_id, mated_score, gan_score, non_mated_score, scoring])  # Store the user ID and mated comparison score
            
            total_comparisons += 1
            if non_mated_score > gan_score:
                num_higher += 1
                        
        except IndexError:
            print(f"No face detected in {img2_path}")
            
                
    print(f"Total number of comparisons: {total_comparisons}")
    print(f"Number of times non-mated score is higher: {num_higher}")
    
# Save the mated comparison scores to a CSV file
with open(output_file_path, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Cluster ID', 'User ID', 'Mated Comparison Score', 'GAN Comparison Score', 'Non-Mated Comparison Score'])  # Write the header row
    writer.writerows(excel_scores)  # Write the user IDs and mated comparison scores

print("Script execution completed.")


Processing cluster 0 with user IDs ['02463', '04397', '04513', '04535', '04601', '04810']
Processing user ID:  02463
Non-Mated Comparison Score: 1.0892371479042586
GAN Comparison Score: 0.922258259622714
Mated Comparison Score: 0.02154193427179174
Processing user ID:  04397
Non-Mated Comparison Score: 0.9450867741247793
GAN Comparison Score: 0.9282575548439884
Mated Comparison Score: 0.10085166401405576
Processing user ID:  04513
Non-Mated Comparison Score: 1.0725789938825268
GAN Comparison Score: 0.9950966479922676
Mated Comparison Score: 0.14692676151494322
Processing user ID:  04535
No face detected in /volumes/1/thomas/thomas-comparison.jpg
Processing user ID:  04601
Non-Mated Comparison Score: 0.8661656266708102
GAN Comparison Score: 0.953329744702706
Mated Comparison Score: 0.4779026798730267
Processing user ID:  04810
Non-Mated Comparison Score: 0.9305563293244508
GAN Comparison Score: 1.0369114110396584
Mated Comparison Score: 0.12907285389578482
Total number of comparisons: 5
